In [ ]:


# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
     
Get started with BigQuery DataFrames
Colab logo Run in Colab	GitHub logo View on GitHub	Vertex AI logo Open in Vertex AI Workbench
NOTE: This notebook has been tested in the following environment:

Python version = 3.10
Overview
Use this notebook to get started with BigQuery DataFrames, including setup, installation, and basic tutorials.

BigQuery DataFrames provides a Pythonic DataFrame and machine learning (ML) API powered by the BigQuery engine.

bigframes.pandas provides a pandas-like API for analytics.
bigframes.ml provides a scikit-learn-like API for ML.
Learn more about BigQuery DataFrames.

Objective
In this tutorial, you learn how to install BigQuery DataFrames, load data into a BigQuery DataFrames DataFrame, and inspect and manipulate the data using pandas and a custom Python function, running at BigQuery scale.

The steps include:

Creating a BigQuery DataFrames DataFrame: Access data from a Parquet file stored in GCS to create a BigQuery DataFrames DataFrame.
Inspecting and manipulating data: Use pandas to perform data cleaning and preparation on the DataFrame.
Deploying a custom function: Deploy a remote function that runs a scalar Python function at BigQuery scale.
Dataset
This tutorial uses the Jump Start Solution dataset (thelook), which contains syntheic retail sales data for a fictional eCommerce clothing retailer.

The same dataset is also deployed to a Cloud Storage bucket in your project as Parquet files so that you can use it to try ingesting data from a local environment.

Costs
This tutorial uses billable components of Google Cloud:

BigQuery (storage and compute)
Cloud Functions
Learn about BigQuery storage pricing, BigQuery compute pricing, and Cloud Functions pricing, and use the Pricing Calculator to generate a cost estimate based on your projected usage.

Before you begin
Complete the tasks in this section to set up your environment.

Set your project ID
If you don't know your project ID, try the following:

Run gcloud config list.
Run gcloud projects list.
See the support page: Locate the project ID.

PROJECT_ID = ${PROJECT_ID}
     
Set the region
You can also change the REGION variable used by BigQuery. Learn more about BigQuery regions.


REGION = ${REGION}
     
Import libraries

import bigframes.pandas as bf
     
Set BigQuery DataFrames options

bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION
     
If you want to reset the location of the created DataFrame or Series objects, reset the session by executing bf.close_session(). After that, you can reuse bf.options.bigquery.location to specify another location.

See the power of BigQuery DataFrames first-hand
BigQuery DataFrames enables you to interact with datasets of any size, so that you can explore, transform, and understand even your biggest datasets using familiar tools like pandas and scikit-learn.

For example, take the BigQuery sample table bigquery-samples.wikipedia_pageviews.200809h, which is ~60 GB is size. This is not a dataset you'd likely be able process in pandas without extra infrastructure.

With BigQuery DataFrames, however, computation is handled by BigQuery's highly scalable compute engine, meaning you can focus on doing data science without hitting size limitations.

If you'd like to try creating a BigQuery DataFrames DataFrame from this table, uncomment and run the next cell to load the table using the read_gbq method.

Note: Keep in mind that running these operations will count against your monthly free tier allowance in BigQuery.


# bq_df_sample = bf.read_gbq("bigquery-samples.wikipedia_pageviews.200809h")
     
No problem! BigQuery DataFrames makes a DataFrame, bq_df_sample, containing the entirety of the source table of data.

Uncomment and run the following cell to see pandas in action over your new BigQuery DataFrames DataFrame.

This code uses regex to filter the DataFrame to include only rows with Wikipedia page titles containing the word "Google", sums the total views by page title, and then returns the top 100 results.


# bq_df_sample[bq_df_sample.title.str.contains(r"[Gg]oogle")]\
# .groupby(['title'], as_index=False)['views'].sum(numeric_only=True)\
# .sort_values('views', ascending=False)\
# .head(100)
     
In addition to giving you access to pandas, BigQuery DataFrames also enables you to build ML models, run inference, and deploy and run your own Python functions at scale. You'll see examples throughout this notebook.

Now you'll move to the JSS dataset (thelook) for the remainder of this getting started guide.

Create a BigQuery DataFrames DataFrame
You can create a BigQuery DataFrames DataFrame by reading data from any of the following locations:

A data file stored in Cloud Storage
Data stored in a BigQuery table
A local data file
An in-memory pandas DataFrame
The following sections show how to use the first two options.

Create a DataFrame from a GCS file
Use these instructions in the following sections to create a BigQuery DataFrames DataFrame from a file stored in Google Cloud Storage.

Define the file path
First, follow the steps below to define the Parquet file URI.


fn_order_items = "${GCS_BUCKET_URI}/thelook-ecommerce/order_items.parquet"
     
Create a DataFrame
Create a BigQuery DataFrames DataFrame from the parquet file:


df_from_gcs = bf.read_parquet(path=fn_order_items)
     
Take a look at the first few rows of the orders DataFrame that was just created:


df_from_gcs.head()
     
Ingest data from a DataFrame to a BigQuery table
BigQuery DataFrames lets you create a BigQuery table from a BigQuery DataFrames DataFrame on-the-fly.

First, create a BigQuery dataset to house the table.


from google.cloud import bigquery
DATASET_ID = "thelook_bigframes"

client = bigquery.Client(project=PROJECT_ID)
dataset = bigquery.Dataset(PROJECT_ID + "." + DATASET_ID)
client.delete_dataset(DATASET_ID, delete_contents=True, not_found_ok=True)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print(f"Dataset {dataset.dataset_id} created.")
     
Dataset thelook_bigframes created.
Next, use the to_gbq method to create a BigQuery table from the DataFrame:


df_from_gcs.to_gbq(PROJECT_ID + "." + DATASET_ID + ".order_items")
     
Create a DataFrame from BigQuery data
You can create a BigQuery DataFrames DataFrame from a BigQuery table by using the read_gbq method and referencing either an entire table or a SQL query.

Create a BigQuery DataFrames DataFrame from the BigQuery table you created in the previous section, and view a few rows:


query_or_table = f"{PROJECT_ID}.{DATASET_ID}.order_items"
bq_df = bf.read_gbq(query_or_table)
bq_df["order_id"] = bq_df["order_id"].astype("string")
bq_df.head()
     
Inspect and manipulate data in BigQuery DataFrames
Using pandas
You can use pandas as you normally would on the BigQuery DataFrames DataFrame, but calculations happen in the BigQuery query engine instead of your local environment. There are 150+ pandas functions supported in BigQuery DataFrames. You can view the list in the documentation.

To see this in action, inspect one of the columns (or series) of the BigQuery DataFrames DataFrame:


bq_df["sale_price"].head(10)
     
Compute the sum of this series to find the total reveneue from all items sold:


total_sales = bq_df["sale_price"].sum()
print(f"total_sales: {total_sales}")
     
Calculate the mean sales_price of all items in an order using the groupby operation to group by order_id:


bq_df[["order_id", "sale_price"]].groupby(
    by=bq_df["order_id"]).mean(numeric_only=True).head()
     
You can confirm that the calculations were run in BigQuery by clicking "Open job" from the previous cells' output. This takes you to the BigQuery console to view the SQL statement and job details.

Using custom functions
Running your own Python functions (or being able to bring your packages) and using them at scale is a challenge many data scientists face. BigQuery DataFrames makes it easy to deploy remote functions that run scalar Python functions at BigQuery scale. These functions are persisted as BigQuery remote functions that you can then re-use.

Running the cell below creates a custom function using the remote_function method. This function categorizes a value into one of two buckets: >= 50 or <50.

Note: Creating a function requires a BigQuery connection. This code assumes a pre-created connection named bigframes-default-connection. If the connection is not already created, BigQuery DataFrames attempts to create one assuming the necessary APIs and IAM permissions are set up in the project.

This cell takes a few minutes to run because it creates the BigQuery connection (if applicable) and deploys the Cloud Function.


@bf.remote_function([float], str)
def get_bucket(num):
  if not num:
    return "NA"
  boundary = 50
  return "at_or_above_50" if num >= boundary else "below_50"
     
The custom function is deployed as a Cloud Function, and is then integrated with BigQuery as a remote function.

Save both of the function names so that you can clean them up at the end of this notebook.


CLOUD_FUNCTION_NAME = format(get_bucket.bigframes_cloud_function)
print("Cloud Function Name " + CLOUD_FUNCTION_NAME)
REMOTE_FUNCTION_NAME = format(get_bucket.bigframes_remote_function)
print("Remote Function Name " + REMOTE_FUNCTION_NAME)
     
Apply the custom function to the BigQuery DataFrames DataFrame to bucketize the sale_price value of each item sold:


bq_df = bq_df.assign(order_price_bucket=bq_df["sale_price"].groupby(
    by=bq_df["order_id"]).sum().apply(get_bucket))
bq_df[["order_id", "order_price_bucket"]].head(10)
     
Summary and next steps
You've created BigQuery DataFrames DataFrames, and inspected and manipulated data with pandas and custom remote functions at BigQuery scale and speed.

Learn more about BigQuery DataFrames in the documentation and find more sample notebooks in the GitHub repo, including an introductory notebook for bigframes.ml.

Cleaning up
To clean up all Google Cloud resources used in this project, you can delete the Google Cloud project you used for the tutorial.

Otherwise, you can uncomment the remaining cells and run them to delete the individual resources you created in this tutorial:


# # Delete the BigQuery dataset
# from google.cloud import bigquery
# client = bigquery.Client(project=PROJECT_ID)
# client.delete_dataset(
#  DATASET_ID, delete_contents=True, not_found_ok=True
# )
# print("Deleted dataset '{}'.".format(DATASET_ID))
     

# # Delete the BigQuery Connection
# from google.cloud import bigquery_connection_v1 as bq_connection
# client = bq_connection.ConnectionServiceClient()
# CONNECTION_ID = f"projects/{PROJECT_ID}/locations/{REGION}/connections/bigframes-default-connection"
# client.delete_connection(name=CONNECTION_ID)
# print("Deleted connection '{}'.".format(CONNECTION_ID))
     

# # Delete the Cloud Function
# ! gcloud functions delete {CLOUD_FUNCTION_NAME} --quiet
     

# # Delete the Remote Function
# REMOTE_FUNCTION_NAME = REMOTE_FUNCTION_NAME.replace(PROJECT_ID + ".", "")
# ! bq rm --routine --force=true {REMOTE_FUNCTION_NAME}
     